In [2]:
import pandas as pd
from flask import Flask, request, jsonify
import threading
import nest_asyncio
import time
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

nest_asyncio.apply()


In [3]:
df = pd.DataFrame({
    "Pianta": ["Girasole", "Orchidea", "Felce", "Cactus", "Bonsai"],
    "Altezza_cm": [150, 40, 60, 25, 30]
})
df.to_csv("piante.csv", index=False)
df


,Pianta,Altezza_cm
0,Girasole,150
1,Orchidea,40
2,Felce,60
3,Cactus,25
4,Bonsai,30


In [4]:
app = Flask("MagiaPiante")

@app.route("/magiaPiante/api/crescita", methods=["POST"])
def crescita():
    try:
        dati = request.get_json()
        potione = int(dati.get("potione", 0))  # percentuale crescita

        piante = pd.read_csv("piante.csv")
        # Crescita proporzionale
        piante["Altezza_cm"] = piante["Altezza_cm"] * (1 + potione/100)

        piante.to_csv("piante_aggiornate.csv", index=False)

        return jsonify({"result": "La magia ha fatto crescere le piante!"})
    except Exception as e:
        print("Errore magico:", e)
        return jsonify({"result": "Errore nell'applicare la magia"})


In [ ]:
from werkzeug.serving import make_server

class ServerThread(threading.Thread):
    def __init__(self, app):
        threading.Thread.__init__(self)
        self.server = make_server('127.0.0.1', 5000, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        print("Starting Flask server...")
        self.server.serve_forever()

    def shutdown(self):
        print("Shutting down Flask server...")
        self.server.shutdown()

# Usa così:
server = ServerThread(app)
server.start()

import time
time.sleep(1)  # lascia partire il server


Starting Flask server...


In [7]:

# ... poi quando vuoi fermare il server:
server.shutdown()

Shutting down Flask server...


In [ ]:
slider = widgets.IntSlider(
    value=0,
    min=-50,
    max=100,
    step=5,
    description='Pozione Crescita (%)',
    continuous_update=False
)

button = widgets.Button(
    description='Applica Pozione',
    button_style='success',
    tooltip='Fai crescere le piante!'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        potione = slider.value
        print(f"Applicando pozione di crescita: {potione}%")
        try:
            risposta = requests.post(
                "http://127.0.0.1:5000/magiaPiante/api/crescita",
                json={"potione": potione}
            )
            print("Risposta server:", risposta.json()["result"])
            dati_agg = pd.read_csv("piante_aggiornate.csv")
            display(dati_agg)
        except Exception as e:
            print("Errore magia:", e)

button.on_click(on_button_clicked)

display(slider, button, output)


IntSlider(value=0, continuous_update=False, description='Pozione Crescita (%)', min=-50, step=5)

Button(button_style='success', description='Applica Pozione', style=ButtonStyle(), tooltip='Fai crescere le pi…

Output()